GitHub repo for this: https://github.com/reuven/2024-09September-08-decorators

# Decorators

1. What are decorators?
2. Writing your first decorator
3. Outer function storage
4. Inputs and outputs
5. Decorators that take arguments
6. Nested decorators
7. Decorating classes
8. Writing decorators as classes

In [1]:
# let's assume that we have two simple functions, a and b, which return something very simple

def a():
    return f'A!\n'

def b():
    return f'B!\n'

In [2]:
print(a())
print(b())

A!

B!



In [3]:
# our company has mandated that we put dashed lines above and below everything we print
# one option is for us to rewrite the functions

lines = '-' * 60 + '\n'

def a():
    return f'{lines}A!\n{lines}'

def b():
    return f'{lines}B!\n{lines}'

print(a())
print(b())

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [5]:
# the problem is that we will need to repeat this for every function we write!

# how can I rewrite this, such that I repeat myself as little as possible?
# option 1: write a function that takes our function as input, and returns the lines + our function's output

lines = '-' * 60 + '\n'

def with_lines(func):
    return f'{lines}{func()}{lines}'

def a():
    return f'A!\n'

def b():
    return f'B!\n'
    
print(with_lines(a))  # I'm not calling a
print(with_lines(b))  # I'm not calling b, either!

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [6]:
# the problem with the above is that we broke the existing/older API
# can we accomplish our goals, while keeping the old API?

# option 2: write a function that returns a function
# this is known as a closure -- we call a function, it returns a function, and then we assign that returned function to a variable

lines = '-' * 60 + '\n'

def with_lines(func):  
    def wrapper():    # closure
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'A!\n'
with_lines_a = with_lines(a)   # once again, I'm not calling a

def b():
    return f'B!\n'
with_lines_b = with_lines(b)  
    
print(with_lines_a())  
print(with_lines_b()) 

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



# How many functions?

1. We have the original `a` and `b` functions
2. We have `with_lines`, which we call, passing `a` and `b` to it
3. We have two functions, `with_lines_a` and `with_lines_b`, that we got back from calling `with_lines` with `a` and `b`

This is fine, but we can do even better -- we can assign the returned function from `with_lines(a)` not to `with_lines_a`, but rather to `a`. Similarly, we can assign the result of `with_lines(b)` back to `b`. As soon as we do that, we don't have direct access to the original `a` and `b` functions.

In [7]:
# option 3: don't define with_lines_a and with_lines_b. Rather, just assign back to a and b
# at this point, we have returned the original API, where we can call a() and b(), to working order

lines = '-' * 60 + '\n'

def with_lines(func):  
    def wrapper():    # closure
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'A!\n'
a = with_lines(a)   # once again, I'm not calling a

def b():
    return f'B!\n'
b = with_lines(b)  
    
print(a())  
print(b()) 

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [8]:
# Finally, we can use Python's decorator syntax
# this is identical in functionality to what we just did and saw, but it looks a bit different


lines = '-' * 60 + '\n'

def with_lines(func):  
    def wrapper():    # closure
        return f'{lines}{func()}{lines}'
    return wrapper

@with_lines    # this line is equivalent to line 15
def a():
    return f'A!\n'
# a = with_lines(a)   

@with_lines   # this is equivalent to line 20
def b():
    return f'B!\n'
# b = with_lines(b)  
    
print(a())  
print(b()) 

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



# Implications of this syntax

1. Don't have both line 12 + 15! There's normally no reason for line 15, but if you're teaching decorators, forgetting to comment out one of them can be not good.
2. If you want to apply the same decorator to a number of different functions, you must say @decorator above each definition.

# Who needs this? Where do we use decorators?

1. If I have several functions that need to do the same thing, I can extract that functionality into a decorator, and simplify my code.
2. Timing of functions
3. Logging of functions
4. Security wrappers
5. Setting values before/after a function runs
6. Check inputs and/or filter them
7. Check outputs and/or filter them

Decorators allow us to hijack a function both when it is defined (then, the outer function is run) and when it is invoked (then, the inner function is run). These are fully fledged functions that we can use however we want.